## **3. Proje Ödevi – Çoklu Doğrusal Regresyon ve Flask Uygulaması**

SHAMS AL HAJJI  
Öğrenci Numarası:  2112721301
GitHub Repository:
https://github.com/ShamsALHAJJI44/Coklu_Dogrusal_Regresyon_Matematik_Notu_Tahmini

Kullanılan Veri Seti: Students Performance Dataset  
Hedef Değişken: Math Score


**1.Giriş**

Bu projede StudentsPerformance veri seti kullanılarak çoklu doğrusal regresyon modeli oluşturulmuştur.
Amaç, öğrencilerin matematik notlarını diğer özellikler (reading score, writing score, lunch, gender vb.) üzerinden tahmin etmektir.
Proje kapsamında veri ön işleme, kategorik değişken kodlama, Backward Elimination yöntemi, model eğitimi, performans değerlendirmesi
ve Flask tabanlı bir web arayüzü geliştirilmiştir.


**2. Kütüphanelerin Yüklenmesi**

In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import statsmodels.api as sm
import pickle


**3.Veri Setinin Yüklenmesi**

 Veri seti StudentsPerformance.csv dosyasından okunmuştur. Veri 1000 öğrencinin

 performans ve demografik bilgilerini içermektedir.


In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Project3/StudentsPerformance.csv")
df.head()


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


**4. Eksik Veri Analizi (Missing Values)**

Veri setindeki eksik değerler kontrol edilmiştir. Bu veri setinde eksik değer bulunmamaktadır.
Bu nedenle silme veya doldurma işlemine ihtiyaç duyulmamıştır.


In [18]:
df.isnull().sum()


,0
gender,0
race/ethnicity,0
parental level of education,0
lunch,0
test preparation course,0
math score,0
reading score,0
writing score,0


**5.Kategorik Değişkenlerin Kodlanması (Encoding)**

gender, race/ethnicity, lunch, parental level of education ve test preparation course değişkenleri kategoriktir.
Bu değişkenlere One-Hot Encoding uygulanmıştır. drop_first=True ile gereksiz dummy tuzağı engellenmiştir.


In [19]:
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded.head()


,math score,reading score,writing score,gender_male,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school,lunch_standard,test preparation course_none
0,72,72,74,False,True,False,False,False,True,False,False,False,False,True,True
1,69,90,88,False,False,True,False,False,False,False,False,True,False,True,False
2,90,95,93,False,True,False,False,False,False,False,True,False,False,True,True
3,47,57,44,True,False,False,False,False,False,False,False,False,False,False,True
4,76,78,75,True,False,True,False,False,False,False,False,True,False,True,True


**6. Feature ve Target Ayrımı**

Hedef değişken (y): math score  
Bağımsız değişkenler (X): diğer tüm özellikler



In [20]:
X = df_encoded.drop("math score", axis=1)
y = df_encoded["math score"]


**7.Backward Elimination (İstatistiksel Özellik Seçimi)**

Backward Elimination yöntemi, istatistiksel olarak anlamsız (p > 0.05) değişkenlerin modelden çıkarılmasını sağlar.
Aşağıdaki OLS regresyon çıktısı kullanılarak yüksek p-değerine sahip değişkenler belirlenmiş ve gerekli sadeleştirme yapılmıştır.


In [21]:
import numpy as np

X_ols = sm.add_constant(np.asarray(X).astype(float))
y_ols = np.asarray(y).astype(float)

model_ols = sm.OLS(y_ols, X_ols).fit()
model_ols.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.877
Model:                            OLS   Adj. R-squared:                  0.875
Method:                 Least Squares   F-statistic:                     500.3
Date:                Thu, 04 Dec 2025   Prob (F-statistic):               0.00
Time:                        22:19:28   Log-Likelihood:                -3090.7
No. Observations:                1000   AIC:                             6211.
Df Residuals:                     985   BIC:                             6285.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.6045      1.245     -9.322      0.000     -14.047      -9.162
x1             0.2635      0.042      6.266      0.000       0.181       0.346
x2             0.7016      0.044     16.120      0.000       0.616       0.787
x3            13.2405      0.372     35.599      0.000      12.511      13.970
x4             0.8354      0.692      1.207      0.228      -0.523       2.194
x5             0.1782      0.649      0.275      0.784      -1.095       1.452
x6             0.0984      0.670      0.147      0.883      -1.217       1.413
x7             5.0777      0.737      6.888      0.000       3.631       6.524
x8            -1.0469      0.616     -1.700      0.089      -2.255       0.161
x9             0.5677      0.535      1.061      0.289      -0.482       1.618
x10           -1.8561      0.793     -2.340      0.019      -3.413      -0.299
x11            0.4003      0.508      0.788      0.431      -0.597       1.397
x12            0.5522      0.550      1.004      0.316      -0.527       1.631
x13            3.2127      0.374      8.585      0.000       2.478       3.947
x14            3.5023      0.397      8.831      0.000       2.724       4.281
==============================================================================
Omnibus:                        0.330   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.848   Jarque-Bera (JB):                0.402
Skew:                          -0.034   Prob(JB):                        0.818
Kurtosis:                       2.930   Cond. No.                         865.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

OLS sonuçlarına göre değişkenlerin büyük çoğunluğu istatistiksel olarak anlamlıdır (p < 0.05).
Bu nedenle modelden çıkarılması gereken bir değişken bulunmamıştır.


**8. Eğitim ve Test Verisine Bölme**

Veri %80 eğitim, %20 test olacak şekilde ayrılmıştır.


In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


**9. Modelin Eğitilmesi**

Çoklu doğrusal regresyon modeli eğitilmiştir.



In [23]:
lr = LinearRegression()
lr.fit(X_train, y_train)


LinearRegression()

**10. Tahmin ve Performans Değerlendirme**

Model performansı R², MAE ve MSE metrikleri ile değerlendirilmiştir.

R² = 0.88 → Model veriyi oldukça iyi açıklamaktadır.

MAE ≈ 4.21 → Ortalama hata yaklaşık 4 puandır.

MSE ≈ 29 → Kare hata oldukça düşüktür.



In [24]:
y_pred = lr.predict(X_test)

print("R2 Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))


R2 Score: 0.8804332983749565
MAE: 4.214763142474851
MSE: 29.095169866715487


**11. Eğitilmiş Modelin Kaydedilmesi**

Flask arayüzünde kullanılmak üzere model pickle formatında kaydedilmiştir.


In [25]:
pickle.dump(lr, open("model.pkl", "wb"))


In [26]:
!cp /content/model.pkl /content/drive/MyDrive/Colab_Notebooks/Project3/


In [27]:
list(X.columns)


['reading score',
 'writing score',
 'gender_male',
 'race/ethnicity_group B',
 'race/ethnicity_group C',
 'race/ethnicity_group D',
 'race/ethnicity_group E',
 "parental level of education_bachelor's degree",
 'parental level of education_high school',
 "parental level of education_master's degree",
 'parental level of education_some college',
 'parental level of education_some high school',
 'lunch_standard',
 'test preparation course_none']

**12. Sonuç**

Bu çalışmada StudentsPerformance veri seti kullanılarak öğrencilerin matematik notlarını tahmin eden bir regresyon modeli geliştirilmiştir.
Model yüksek doğruluk (R² = 0.88) sağlamıştır.  
Bir sonraki aşamada Flask tabanlı bir web arayüzü geliştirilerek kullanıcıların veri girişi yaparak matematik notu tahmini alması sağlanmıştır.
